# Dateioperationen und I/O

Dieses Notebook zeigt den Einstieg in echte Dateien auf der Festplatte.

## Mentales Modell
- Eine Datei ist wie ein Textstrom.
- Der Modus (`r`, `w`, `a`, `x`) ist deine Absicht.

## Lernstufen
- Pflicht: `with open(...)`, Modi verstehen, Pfade sicher bauen
- Nuetzlich: Fehlerbehandlung, zeilenweise Verarbeitung, Encoding
- Spaeter: groessere Projekte mit vielen Dateien


## 0) Jupyter-Realitaet: Wo bin ich?

Bevor du mit Dateien arbeitest, pruefe immer:
- aktuelles Arbeitsverzeichnis (`cwd`)
- welche Dateien/Ordner dort liegen


In [ ]:
import os
from pathlib import Path

print("Aktuelles Verzeichnis:")
print(os.getcwd())

print("\nErste Eintraege im aktuellen Ordner:")
for name in os.listdir()[:10]:
    print("-", name)


In [ ]:
# Demo-Ordner fuer dieses Notebook
base_dir = Path("io_demo")
base_dir.mkdir(exist_ok=True)
print("Demo-Ordner:", base_dir.resolve())


### Mini-Checkpoint

- Frage: Warum ist `cwd` wichtig bei Datei-Fehlern?
- Mini-Aufgabe: Pruefe, ob `io_demo` existiert.


In [ ]:
print((Path("io_demo")).exists())


## 1) Datei-Modi als Spickzettel (Pflicht)

Datei-Modi:
- `r` = lesen (Datei muss existieren)
- `w` = schreiben (ueberschreibt komplett)
- `a` = anhaengen (hinten einfuegen)
- `x` = neu erstellen (Fehler, wenn Datei existiert)
- `b` = binaer (spaeter)

Wichtige Warnung:
`w` loescht den alten Inhalt der Datei.


## 2) Pflichtregel: fast immer `with open(...) as f`

Merksatz:
Nutze als Standard immer `with`.
Dann wird die Datei automatisch geschlossen.


In [ ]:
datei = base_dir / "regel_with.txt"

with open(datei, "w", encoding="utf-8") as f:
    f.write("Mit with geschrieben.\n")

print("Datei geschlossen:", f.closed)


In [ ]:
# Auch bei Fehlern schliesst with die Datei sauber
fehler_datei = base_dir / "with_fehler_demo.txt"

try:
    with open(fehler_datei, "w", encoding="utf-8") as f:
        f.write("Zeile 1\n")
        raise RuntimeError("Absichtlicher Testfehler")
except RuntimeError as e:
    print("Fehler abgefangen:", e)

print("Datei geschlossen nach Fehler:", f.closed)


### Deine Zelle

- Schreibe eine Datei `hello.txt` im Demo-Ordner mit einer Zeile.
- Lies sie direkt danach wieder ein.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
pfad = base_dir / "hello.txt"
with open(pfad, "w", encoding="utf-8") as f:
    f.write("Hallo Datei!\n")

with open(pfad, "r", encoding="utf-8") as f:
    print(f.read())


### Mini-Checkpoint

- Frage: Warum ist `with` sicherer als manuelles `open/close`?
- Mini-Aufgabe: Pruefe mit `f.closed`, ob die Datei nach `with` geschlossen ist.


In [ ]:
pfad = base_dir / "check_closed.txt"
with open(pfad, "w", encoding="utf-8") as f:
    f.write("ok")
print(f.closed)


## 3) Modi praktisch: `r`, `w`, `a`, `x`


In [ ]:
modus_datei = base_dir / "modi_demo.txt"

# w: neu schreiben / ueberschreiben
with open(modus_datei, "w", encoding="utf-8") as f:
    f.write("Start\n")

with open(modus_datei, "r", encoding="utf-8") as f:
    print("Nach w:\n" + f.read())


In [ ]:
# a: anhaengen
with open(modus_datei, "a", encoding="utf-8") as f:
    f.write("Anhang 1\n")
    f.write("Anhang 2\n")

with open(modus_datei, "r", encoding="utf-8") as f:
    print("Nach a:\n" + f.read())


In [ ]:
# x: nur neu erstellen
x_datei = base_dir / "nur_einmal.txt"

try:
    with open(x_datei, "x", encoding="utf-8") as f:
        f.write("Erstellt mit x\n")
    print("x-Datei erstellt")
except FileExistsError:
    print("x-Datei existiert bereits")


In [ ]:
# r: lesen -> Datei muss existieren
nicht_da = base_dir / "nicht_da.txt"

try:
    with open(nicht_da, "r", encoding="utf-8") as f:
        print(f.read())
except FileNotFoundError as e:
    print("Datei nicht gefunden:", e)


### Deine Zelle

- Erstelle `tagebuch.txt` mit `w`.
- Fuege mit `a` eine zweite Zeile hinzu.
- Lies den Inhalt mit `r`.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
tagebuch = base_dir / "tagebuch.txt"

with open(tagebuch, "w", encoding="utf-8") as f:
    f.write("Tag 1: Start\n")

with open(tagebuch, "a", encoding="utf-8") as f:
    f.write("Tag 2: Weiter geht's\n")

with open(tagebuch, "r", encoding="utf-8") as f:
    print(f.read())


### Mini-Checkpoint

- Frage: Wann ist `a` besser als `w`?
- Mini-Aufgabe: Erklaere in einem Satz den Unterschied zwischen `x` und `w`.


In [ ]:
print("w ueberschreibt; x erstellt nur, wenn die Datei noch nicht existiert.")


## 4) Lesen: drei typische Muster

Muster 1: ganze Datei (klein)
Muster 2: zeilenweise (haeufig)
Muster 3: alle Zeilen als Liste


In [ ]:
lese_datei = base_dir / "lesen_demo.txt"
with open(lese_datei, "w", encoding="utf-8") as f:
    f.write("INFO Start\n")
    f.write("WARNING Speicher knapp\n")
    f.write("ERROR Datei fehlt\n")


In [ ]:
# Muster 1: komplette Datei
with open(lese_datei, "r", encoding="utf-8") as f:
    text = f.read()

print(text)


In [ ]:
# Stolperfalle: read() zweimal
with open(lese_datei, "r", encoding="utf-8") as f:
    erst = f.read()
    zweit = f.read()

print("Erster read hat Laenge:", len(erst))
print("Zweiter read ist leer:", repr(zweit))


In [ ]:
# Muster 2: zeilenweise
with open(lese_datei, "r", encoding="utf-8") as f:
    for zeile in f:
        print("Zeile:", zeile.strip())


In [ ]:
# Muster 3: readlines()
with open(lese_datei, "r", encoding="utf-8") as f:
    zeilen = f.readlines()

print("Liste von Zeilen:", zeilen)
print("Zeilenanzahl:", len(zeilen))


### Deine Zelle

- Zaehle die Zeilen in `lesen_demo.txt`.
- Zaehle, wie oft das Wort `ERROR` vorkommt.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
zeilenzahl = 0
errors = 0

with open(lese_datei, "r", encoding="utf-8") as f:
    for zeile in f:
        zeilenzahl += 1
        if "ERROR" in zeile:
            errors += 1

print("Zeilen:", zeilenzahl)
print("ERROR:", errors)
assert zeilenzahl == 3
assert errors == 1


### Mini-Checkpoint

- Frage: Warum ist `for zeile in f` oft der beste Lesemodus?
- Mini-Aufgabe: Gib nur Zeilen mit `WARNING` aus.


In [ ]:
with open(lese_datei, "r", encoding="utf-8") as f:
    for zeile in f:
        if "WARNING" in zeile:
            print(zeile.strip())


## 5) Schreiben: `write` und `writelines`

Merksatz:
- `write` schreibt einen String
- `writelines` schreibt viele Strings

Wichtig:
`writelines` fuegt keine `\n` automatisch hinzu.


In [ ]:
ziel = base_dir / "schreiben_demo.txt"

with open(ziel, "w", encoding="utf-8") as f:
    n = f.write("Zeile A\n")

print("Geschriebene Zeichen:", n)


In [ ]:
# writelines ohne \n -> alles klebt zusammen
ohne_nl = base_dir / "writelines_ohne_nl.txt"
with open(ohne_nl, "w", encoding="utf-8") as f:
    f.writelines(["eins", "zwei", "drei"])

with open(ohne_nl, "r", encoding="utf-8") as f:
    print(f.read())


In [ ]:
# writelines mit \n
mit_nl = base_dir / "writelines_mit_nl.txt"
with open(mit_nl, "w", encoding="utf-8") as f:
    f.writelines(["eins\n", "zwei\n", "drei\n"])

with open(mit_nl, "r", encoding="utf-8") as f:
    print(f.read())


### Deine Zelle

- Schreibe 3 Zeilen in `drei_zeilen.txt`, jede in neuer Zeile.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
pfad = base_dir / "drei_zeilen.txt"
with open(pfad, "w", encoding="utf-8") as f:
    for i in range(1, 4):
        f.write(f"Zeile {i}\n")

with open(pfad, "r", encoding="utf-8") as f:
    text = f.read()

print(text)
assert "Zeile 1" in text and "Zeile 3" in text


### Mini-Checkpoint

- Frage: Wann ist `writelines` praktisch?
- Mini-Aufgabe: Schreibe nur Zeilen mit `ERROR` aus `lesen_demo.txt` in `errors.txt`.


In [ ]:
errors_pfad = base_dir / "errors.txt"
with open(lese_datei, "r", encoding="utf-8") as src, open(errors_pfad, "w", encoding="utf-8") as dst:
    for zeile in src:
        if "ERROR" in zeile:
            dst.write(zeile)

with open(errors_pfad, "r", encoding="utf-8") as f:
    print(f.read())


## 6) Pfade robust mit `pathlib` (Pflicht)

`pathlib` ist einsteigerfreundlich und plattformunabhaengig.

Statt String-Verkettung:
- besser: `Path` und `/` Operator


In [ ]:
from pathlib import Path

base = Path.cwd()
daten_ordner = base / "io_demo" / "daten"
daten_ordner.mkdir(parents=True, exist_ok=True)

bericht = daten_ordner / "bericht.txt"
bericht.write_text("Bericht Zeile 1\nBericht Zeile 2\n", encoding="utf-8")

print("Datei:", bericht)
print("Absolut:", bericht.resolve())
print("Existiert:", bericht.exists())
print("Ist Datei:", bericht.is_file())
print("Endung:", bericht.suffix)
print("Dateiname:", bericht.name)
print("Ordnername:", bericht.parent.name)


In [ ]:
print("Eintraege in io_demo:")
for p in sorted((Path("io_demo")).iterdir()):
    art = "Ordner" if p.is_dir() else "Datei"
    print(f"- {p.name:30s} ({art})")


### Deine Zelle

- Erstelle mit `Path` einen Unterordner `berichte`.
- Speichere darin `heute.txt`.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
ordner = Path("io_demo") / "berichte"
ordner.mkdir(exist_ok=True)

heute = ordner / "heute.txt"
heute.write_text("Tagesbericht\n", encoding="utf-8")
print(heute.resolve())


### Mini-Checkpoint

- Frage: Warum ist `pathlib` besser als manuelles Pfad-Zusammenkleben?
- Mini-Aufgabe: Gib alle Dateien (ohne Ordner) in `io_demo` aus.


In [ ]:
for p in (Path("io_demo")).iterdir():
    if p.is_file():
        print(p.name)


## 7) Fehlerbehandlung bei Datei-I/O (Pflicht)

Typische Fehler:
- Datei nicht gefunden
- kein Zugriff
- allgemeiner OSError

Regel:
Nicht `except:` blind nutzen. Konkrete Fehler abfangen.


In [ ]:
def lese_datei_sicher(pfad: Path):
    try:
        with open(pfad, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"Datei nicht gefunden: {pfad}")
        return None
    except PermissionError:
        print(f"Keine Berechtigung fuer: {pfad}")
        return None
    except OSError as e:
        print(f"Allgemeiner Dateifehler: {e}")
        return None

print(lese_datei_sicher(Path("io_demo") / "nicht_vorhanden.txt"))


In [ ]:
def schreibe_datei_sicher(pfad: Path, text: str):
    try:
        pfad.parent.mkdir(parents=True, exist_ok=True)
        with open(pfad, "w", encoding="utf-8") as f:
            f.write(text)
        return True
    except OSError as e:
        print("Schreibfehler:", e)
        return False

ok = schreibe_datei_sicher(Path("io_demo") / "safe" / "ausgabe.txt", "Hallo safe\n")
print("Erfolg:", ok)


### Deine Zelle

- Schreibe eine Funktion, die eine Datei liest und bei Fehler `""` zurueckgibt.


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
def lese_oder_leer(pfad: Path):
    try:
        with open(pfad, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return ""

print(repr(lese_oder_leer(Path("io_demo") / "x.txt")))


### Mini-Checkpoint

- Frage: Warum ist `except FileNotFoundError` besser als `except:`?
- Mini-Aufgabe: Lies eine bestehende Datei mit deiner sicheren Funktion.


In [ ]:
print(lese_datei_sicher(Path("io_demo") / "modi_demo.txt")[:20])


## 8) Encoding kurz und praktisch

Wenn Umlaute kaputt aussehen, setze `encoding="utf-8"`.


In [ ]:
utf_datei = base_dir / "umlaute.txt"
inhalt = "Käse, Grüße, Köln\n"

with open(utf_datei, "w", encoding="utf-8") as f:
    f.write(inhalt)

with open(utf_datei, "r", encoding="utf-8") as f:
    print(f.read())


In [ ]:
# Falsches Encoding demonstrieren
try:
    with open(utf_datei, "r", encoding="ascii") as f:
        print(f.read())
except UnicodeDecodeError as e:
    print("UnicodeDecodeError:", e)


### Mini-Checkpoint

- Frage: Wann brauchst du explizit `encoding="utf-8"`?
- Mini-Aufgabe: Schreibe und lies einen Text mit Umlauten.


In [ ]:
# Deine Zelle



## 9) Mini-Projekt: Mini-Log-Analyzer

Aufgabe:
- Log-Datei lesen
- Anzahl Zeilen zaehlen
- ERROR/WARNING zaehlen
- Summary in Datei schreiben
- ERROR-Zeilen in `errors.txt` filtern


In [ ]:
log_datei = base_dir / "app.log"

log_zeilen = [
    "INFO Start",
    "INFO Benutzer eingeloggt",
    "WARNING Speicher fast voll",
    "ERROR Datei fehlt",
    "INFO Wiederholung",
    "ERROR Netzwerkfehler",
]

with open(log_datei, "w", encoding="utf-8") as f:
    for z in log_zeilen:
        f.write(z + "\n")

print("Logdatei erstellt:", log_datei)


In [ ]:
def lese_logzeilen(pfad: Path):
    with open(pfad, "r", encoding="utf-8") as f:
        return [z.strip() for z in f]


def analysiere_log(zeilen):
    stats = {
        "gesamt": len(zeilen),
        "error": sum(1 for z in zeilen if "ERROR" in z),
        "warning": sum(1 for z in zeilen if "WARNING" in z),
    }
    return stats


def schreibe_summary(pfad: Path, stats: dict):
    with open(pfad, "w", encoding="utf-8") as f:
        f.write(f"Gesamt: {stats['gesamt']}\n")
        f.write(f"ERROR: {stats['error']}\n")
        f.write(f"WARNING: {stats['warning']}\n")


def schreibe_error_zeilen(pfad: Path, zeilen):
    with open(pfad, "w", encoding="utf-8") as f:
        for z in zeilen:
            if "ERROR" in z:
                f.write(z + "\n")


zeilen = lese_logzeilen(log_datei)
stats = analysiere_log(zeilen)
summary_datei = base_dir / "summary.txt"
errors_only = base_dir / "errors_only.txt"

schreibe_summary(summary_datei, stats)
schreibe_error_zeilen(errors_only, zeilen)

print("Stats:", stats)
print("Summary-Datei:", summary_datei)
print("Errors-Datei:", errors_only)


In [ ]:
# Mini-Selbsttests
zeilen = lese_logzeilen(log_datei)
stats = analysiere_log(zeilen)

assert stats["gesamt"] == 6
assert stats["error"] == 2
assert stats["warning"] == 1

print("Mini-Tests ok")


### Deine Zelle (Mini-Projekt)

- Erweitere den Analyzer:
1. zaehle INFO-Zeilen
2. gib den Anteil ERROR in Prozent aus


In [ ]:
# Deine Zelle



In [ ]:
# Loesung (optional)
zeilen = lese_logzeilen(log_datei)
stats = analysiere_log(zeilen)

info_count = sum(1 for z in zeilen if "INFO" in z)
error_quote = stats["error"] / stats["gesamt"] * 100

print("INFO:", info_count)
print("ERROR-%:", round(error_quote, 2))


### Mini-Checkpoint

- Frage: Welche Schritte im Analyzer sind Lesen, Auswerten, Schreiben?
- Mini-Aufgabe: Oeffne `summary.txt` und gib den Inhalt aus.


In [ ]:
with open(base_dir / "summary.txt", "r", encoding="utf-8") as f:
    print(f.read())


## 10) Typische Einsteigerfehler

1. Falsches Arbeitsverzeichnis (`cwd`) vergessen
2. `w` ueberschreibt unbemerkt Dateien
3. Pfade als String zusammenkleben
4. `encoding` vergessen
5. ohne `with` arbeiten und Datei offen lassen
6. `read()` zweimal -> beim zweiten Mal leer
7. Zeilen mit `
` nicht mit `strip()` bereinigen
8. relativer vs. absoluter Pfad unklar
9. `os.listdir()` zeigt auch Ordner, nicht nur Dateien


In [ ]:
print("Fehlercheck-Liste gelesen")


## 11) Uebungen (kurz und haeufig)

### Uebung A
- Zaehle Zeilen in einer Datei.

### Uebung B
- Finde ein Wort in jeder Zeile und zaehle Treffer.

### Uebung C
- Schreibe nur Zeilen mit `ERROR` in neue Datei.

### Uebung D
- Haenge Log-Eintraege mit Modus `a` an.

### Uebung E
- Mini-CSV ohne csv-Modul: split mit `;` und summiere Zahlen.


In [ ]:
# Platz fuer eigene Loesungen



## 12) Spickzettel

```python
# Standard
with open("datei.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Modi
# r lesen | w schreiben (ueberschreibt) | a anhaengen | x neu erstellen

# Pfade mit pathlib
from pathlib import Path
pfad = Path.cwd() / "data" / "test.txt"

# Fehlerbehandlung
try:
    ...
except FileNotFoundError:
    ...

# Struktur zuerst pruefen
# cwd, exists(), is_file(), len(), type()
```


## 13) Aufraeumen (optional)

Wenn du alle Demo-Dateien loeschen willst, fuehre die folgende Zelle aus.


In [ ]:
# Optional ausfuehren
# import shutil
# from pathlib import Path
#
# ziel = Path("io_demo")
# if ziel.exists() and ziel.is_dir():
#     shutil.rmtree(ziel)
#     print("io_demo entfernt")
# else:
#     print("io_demo nicht gefunden")


## Zusammenfassung

- Dateimodus = Absicht (`r`, `w`, `a`, `x`).
- Standardregel: fast immer `with open(...)`.
- Nutze die drei Lesemuster je nach Fall (`read`, zeilenweise, `readlines`).
- Baue Pfade robust mit `pathlib`.
- Fange Dateifehler konkret ab (`FileNotFoundError`, ...).
- Mit kleinen Pipelines (lesen -> auswerten -> schreiben) loest du viele Praxisaufgaben.
